# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load csv file
csv_path = "Outputs/city_data.csv"
city_data = pd.read_csv(csv_path)
city_data.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country Code,Date
0,0,Rikitea,-23.1203,-134.9692,73.53,71,0,14.00,PF,1632235966
1,1,Kankaanpää,61.8000,22.4167,48.96,55,34,1.79,FI,1632235966
2,2,Bima,-8.4667,118.7167,76.86,84,90,1.68,ID,1632235967
3,3,Geraldton,-28.7667,114.6000,52.21,87,0,5.75,AU,1632235967
4,4,Hobart,-42.8794,147.3294,53.46,76,98,4.00,AU,1632235968


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Create map with humidity layer
fig = gmaps.figure()
locations=city_data[['Latitude','Longitude']]
humidity=city_data['Humidity (%)']

humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Create dataframe with ideal weather conditions (temperature between 70 and 80 F, 0% cloud cover, <10 mph wind speeds)
ideal_weather = city_data.loc[(city_data['Max Temp. (F)'] > 70) & (city_data['Max Temp. (F)'] < 80) & (city_data['Wind Speed (mph)'] <10) & (city_data['Cloudiness (%)'] == 0)]

#Drop rows with null values
ideal_weather.dropna()
ideal_weather

,Unnamed: 0,City,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country Code,Date
31,31,Ambilobe,-13.2000,49.0500,78.04,61,0,4.70,MG,1632235979
53,53,Fare,-16.7000,-151.0167,78.12,72,0,9.08,PF,1632235989
75,75,Wenling,28.3669,121.3600,74.97,90,0,3.89,CN,1632235999
247,247,Zabul Province,32.2500,67.2500,72.54,12,0,9.44,AF,1632236075
323,323,San Quintín,30.4833,-115.9500,73.08,69,0,5.12,MX,1632236106
331,331,Yangi Marg`ilon,40.4272,71.7189,75.92,27,0,2.30,UZ,1632236109
399,399,Tevaitoa,-16.7833,-151.5000,78.03,70,0,8.88,PF,1632236139
426,426,Pacific Grove,36.6177,-121.9166,73.27,60,0,1.01,US,1632236149
512,512,Huilong,31.8111,121.6550,76.98,78,0,8.72,CN,1632236185


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather[['City', 'Latitude', 'Longitude','Max Temp. (F)', 'Humidity (%)', 'Cloudiness (%)', 'Wind Speed (mph)', 'Country Code']]
hotel_df['Hotel Name'] = " "
hotel_df.head()

C:\Users\Katli\.conda\envs\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country Code,Hotel Name
31,Ambilobe,-13.2000,49.0500,78.04,61,0,4.70,MG,
53,Fare,-16.7000,-151.0167,78.12,72,0,9.08,PF,
75,Wenling,28.3669,121.3600,74.97,90,0,3.89,CN,
247,Zabul Province,32.2500,67.2500,72.54,12,0,9.44,AF,
323,San Quintín,30.4833,-115.9500,73.08,69,0,5.12,MX,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the type "lodging" and lat/lng coordinates
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params).json()

    # Skip location if there is no loding within 5,000 meters
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        pass
    
hotel_df.head()

C:\Users\Katli\.conda\envs\venv\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Latitude,Longitude,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country Code,Hotel Name
31,Ambilobe,-13.2000,49.0500,78.04,61,0,4.70,MG,Hôtel & Restaurant DIANA Ambilobe
53,Fare,-16.7000,-151.0167,78.12,72,0,9.08,PF,Maitai Lapita Village Huahine
75,Wenling,28.3669,121.3600,74.97,90,0,3.89,CN,Hanting Express Wenling Wanchang North Road
247,Zabul Province,32.2500,67.2500,72.54,12,0,9.44,AF,
323,San Quintín,30.4833,-115.9500,73.08,69,0,5.12,MX,Don Eddie's Sport Fishing Center


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_h = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Create map with humidity layer
fig = gmaps.figure()
locations=city_data[['Latitude','Longitude']]
humidity=city_data['Humidity (%)']

humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(humidity_layer)

# Add marker layer
markers = gmaps.marker_layer(locations_h,info_box_content=hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))